In [26]:
!pip install rdflib
!pip install owlready2
!pip install owlrl
from rdflib import Graph, Namespace, RDF, Literal,RDFS, BNode, XSD
from rdflib.plugins.sparql import prepareQuery
import owlrl

In [27]:
# Load the ontology file
g = Graph()
g.parse('/content/moviesV4.rdf')

<Graph identifier=Nafd47964531d420f90a2f52f428b9b17 (<class 'rdflib.graph.Graph'>)>

In [28]:
# Define the namespaces used in the ontology
moviesV1 = Namespace("http://www.semanticweb.org/dataset/ontologies/2024/4/moviesV1#")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

# **Task 1: Display all Persons without Query or Inference**

In [29]:
persons = set()

# Iterate through all triples in the graph and add Actors, Directors, and Writers to the set
for subj, pred, obj in g:
    if pred == RDF.type and (obj == moviesV1.Actor or obj == moviesV1.Director or obj == moviesV1.Writer):
        persons.add(str(subj))

for person in persons:
    name = person.split('#')[-1]
    print(f"Person: {name}")


Person: Quentin_Tarantino
Person: Frank_Darabont
Person: Bradley_Cooper
Person: Tim_Robbins
Person: Jon_Lucas
Person: Taika_Waititi
Person: Matthew_McConaughey
Person: Christian_Bale
Person: Todd_Phillips
Person: Paul_Thomas_Anderson
Person: Uma_Thurman
Person: Francis_Ford_Coppola
Person: Leonardo_DiCaprio
Person: Marlon_Brando
Person: John_Travolta
Person: Edgar_Wright
Person: Stephen_King
Person: Mario_Puzo
Person: Christopher_Nolan


# **Task 2: Display all Persons by Query only**

In [30]:
query = """
    SELECT DISTINCT ?person WHERE {
        { ?person rdf:type moviesV1:Actor . }
        UNION
        { ?person rdf:type moviesV1:Director . }
        UNION
        { ?person rdf:type moviesV1:Writer . }
    }
"""

# Execute the query
results = g.query(query)

# Print the results
for row in results:
  name = str(row[0]).split('#')[-1]
  print(f"Actor: {name}")


Actor: Bradley_Cooper
Actor: Christian_Bale
Actor: Edgar_Wright
Actor: John_Travolta
Actor: Leonardo_DiCaprio
Actor: Marlon_Brando
Actor: Matthew_McConaughey
Actor: Paul_Thomas_Anderson
Actor: Quentin_Tarantino
Actor: Taika_Waititi
Actor: Tim_Robbins
Actor: Uma_Thurman
Actor: Christopher_Nolan
Actor: Francis_Ford_Coppola
Actor: Frank_Darabont
Actor: Todd_Phillips
Actor: Jon_Lucas
Actor: Mario_Puzo
Actor: Stephen_King


# **Task 3: Display all Persons by Inference only**

In [31]:
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(g)

# Display all the Actors
for subj, pred, obj in g.triples((None, RDF.type, moviesV1.Actor)):
    name = subj.split('#')[-1]
    print(f"Person: {name}")

Person: Bradley_Cooper
Person: Christian_Bale
Person: Edgar_Wright
Person: John_Travolta
Person: Leonardo_DiCaprio
Person: Marlon_Brando
Person: Matthew_McConaughey
Person: Paul_Thomas_Anderson
Person: Quentin_Tarantino
Person: Taika_Waititi
Person: Tim_Robbins
Person: Uma_Thurman
Person: Thor:Ragnarok


# **Task 4: Display Movie properties if Movie Exists**

In [32]:
# Read the name of the movie
movie_name = input("Enter a movie name: ")
movie = moviesV1[movie_name]

# Check if the movie exists in the ontology
if (movie, None, None) not in g:
    print("Error: Movie does not exist")
else:
    # Get and display the movie's year, country, genres, and actors
    year = g.value(movie, moviesV1.year)
    country = g.value(movie, moviesV1.country)
    genres = [str(obj).split('#')[-1] for obj in g.objects(movie, moviesV1.hasGenre)]
    actors = [str(obj).split('#')[-1] for obj in g.objects(movie, moviesV1.hasActor)]
    print(f"Year: {year}, Country: {country}, Genres: {genres}, Actors: {actors}")

Enter a movie name: Thor:Ragnarok
Year: 2017, Country: USA, Genres: ['Action'], Actors: ['Taika_Waititi']


# **Task 5: Add a Rule for a New Class, ActorDirector**

In [33]:
# Create a new class ActorDirector
ActorDirector = BNode()
g.add((ActorDirector, RDF.type, RDFS.Class))
g.add((ActorDirector, RDFS.label, Literal("ActorDirector")))

# Find all persons that are both actors and directors
for person in g.subjects(RDF.type, moviesV1.Person):
    is_actor = (person, moviesV1.isActorOf, None) in g
    is_director = (person, moviesV1.isDirectorOf, None) in g
    if is_actor and is_director:
        # Add the person to the ActorDirector class
        g.add((person, RDF.type, ActorDirector))

# Display all ActorDirectors
for actor_director in g.subjects(RDF.type, ActorDirector):
    ad = str(actor_director).split('#')[-1]
    print(f"ActorDirector: {ad}")


ActorDirector: Paul_Thomas_Anderson
ActorDirector: Edgar_Wright
ActorDirector: Taika_Waititi
ActorDirector: Quentin_Tarantino


# **Task 6: Add New Rules**


In [34]:
# Rule 1: If a person is a writer and a director, they are a WriterDirector
WriterDirector = BNode()
g.add((WriterDirector, RDF.type, RDFS.Class))
g.add((WriterDirector, RDFS.label, Literal("WriterDirector")))

for person in g.subjects(RDF.type, moviesV1.Person):
    is_writer = (person, moviesV1.isWriterOf, None) in g
    is_director = (person, moviesV1.isDirectorOf, None) in g
    if is_writer and is_director:
        g.add((person, RDF.type, WriterDirector))

# Rule 2: If a movie has more than one genre, it is a MultiGenreMovie.
MultiGenreMovie = BNode()
g.add((MultiGenreMovie, RDF.type, RDFS.Class))
g.add((MultiGenreMovie, RDFS.label, Literal("MultiGenreMovie")))

for movie in g.subjects(RDF.type, moviesV1.Movie):
    genres = list(g.objects(movie, moviesV1.hasGenre))
    if len(genres) > 1:
        g.add((movie, RDF.type, MultiGenreMovie))

# Rule 3: If a movie is made after 2000,it is a ModernMovie.
ModernMovie = BNode()
g.add((ModernMovie, RDF.type, RDFS.Class))
g.add((ModernMovie, RDFS.label, Literal("ModernMovie")))
for subj, obj in g.subject_objects(moviesV1.year):
    if int(obj) > 2000:
        g.add((subj, RDF.type, ModernMovie))

print("WriterDirectors:")
for writer_director in g.subjects(RDF.type, WriterDirector):
    wd = str(writer_director).split('#')[-1]
    print(f"WriterDirector: {wd}")

print("\nMultiGenreMovies:")
# Display all MultiGenreMovies
for multi_genre_movie in g.subjects(RDF.type, MultiGenreMovie):
    mg = str(multi_genre_movie).split('#')[-1]
    print(f"MultiGenreMovie: {mg}")

print("\nModernMovies:")
for s in g.subjects(RDF.type, ModernMovie):
    mm = str(s).split('#')[-1]
    print(f"ModernMovie: {mm}")


WriterDirectors:
WriterDirector: Paul_Thomas_Anderson
WriterDirector: Edgar_Wright
WriterDirector: Taika_Waititi
WriterDirector: Christopher_Nolan
WriterDirector: Quentin_Tarantino

MultiGenreMovies:
MultiGenreMovie: Baby_Driver
MultiGenreMovie: Boogie_Nights
MultiGenreMovie: Inception
MultiGenreMovie: Kill_Bill
MultiGenreMovie: Pulp_Fiction
MultiGenreMovie: Shaun_of__the_Dead
MultiGenreMovie: The_Dark_Knight
MultiGenreMovie: The_Godfather

ModernMovies:
ModernMovie: Baby_Driver
ModernMovie: Thor:Ragnarok
ModernMovie: Inception
ModernMovie: Interstellar
ModernMovie: Jojo_Rabbit
ModernMovie: Kill_Bill
ModernMovie: Shaun_of__the_Dead
ModernMovie: The_Dark_Knight
ModernMovie: The_Hangover
ModernMovie: There_Will_Be_Blood
